In [1]:
import pandas as pd
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
path = "/content/drive/MyDrive/UOS_빅데이터경진대회/data/"

gu = pd.read_csv(path+"서울시_자치구_중심점.csv", encoding="EUC-KR")
gu = gu.iloc[:,2:]
gu.columns = ["gu", "X","Y"]

outplace = pd.read_csv(path+"실외 문화생활 정보.csv")
out_Seoul = outplace.query("CTPRVN_NM == '서울특별시'")
out_Seoul = out_Seoul[["LCLAS_NM", "MLSFC_NM", "FCLTY_NM", "CTPRVN_NM", "SIGNGU_NM", "FCLTY_ROAD_NM_ADDR", "FCLTY_LA", "FCLTY_LO"]]
out_Seoul = out_Seoul.reset_index(drop=True)

inplace = pd.read_csv(path+"실내 문화생활 정보.csv")
in_Seoul = inplace.query("CTPRVN_NM == '서울특별시'")
in_Seoul = in_Seoul[["LCLAS_NM", "MLSFC_NM", "FCLTY_NM", "CTPRVN_NM", "SIGNGU_NM", "FCLTY_ROAD_NM_ADDR", "FCLTY_LA", "FCLTY_LO"]]
in_Seoul = in_Seoul.reset_index(drop=True)

tour = pd.read_csv(path + "장애인_숙박관광.csv")
seoul_tour = tour.query("ctprvn_klang_nm == '서울특별시'").reset_index(drop = True)
tour_data = seoul_tour[["se_nm", "fclty_nm", "rn_addr", "ctprvn_klang_nm", "signgn_klang_nm", "lo", "la"]]

In [13]:
dic = {}
gu_li = []
num_li = []
for i in out_Seoul["SIGNGU_NM"].unique():
    gu_li.append(i)
    num_li.append(sum(out_Seoul["SIGNGU_NM"]==i))

dic["gu"] = gu_li
dic["num"] = num_li

In [14]:
out_num = pd.DataFrame(dic)
out_num

,gu,num
0,광진구,17
1,강북구,3
2,강남구,12
3,양천구,14
4,강동구,8
5,강서구,3
6,종로구,55
7,마포구,16
8,영등포구,18
9,노원구,9


In [15]:
dic = {}
gu_li = []
num_li = []
for i in in_Seoul["SIGNGU_NM"].unique():
    gu_li.append(i)
    num_li.append(sum(in_Seoul["SIGNGU_NM"]==i))

dic["gu"] = gu_li
dic["num"] = num_li

In [16]:
in_num = pd.DataFrame(dic)
in_num

,gu,num
0,강남구,88
1,강서구,24
2,송파구,44
3,강동구,17
4,노원구,15
5,관악구,12
6,은평구,10
7,마포구,36
8,광진구,17
9,종로구,135


In [17]:
dic = {}
gu_li = []
num_li = []
for i in tour_data["signgn_klang_nm"].unique():
    gu_li.append(i)
    num_li.append(sum(tour_data["signgn_klang_nm"]==i))

dic["gu"] = gu_li
dic["num"] = num_li

In [18]:
tour_num = pd.DataFrame(dic)
tour_num

,gu,num
0,동작구,6
1,종로구,37
2,중구,57
3,성동구,17
4,강남구,9
5,관악구,8
6,구로구,19
7,마포구,18
8,서대문구,9
9,서초구,6


In [77]:
gu["Y1"] = gu["Y"]
gu["Y2"] = gu["Y"] + 0.001
gu["Y3"] = gu["Y"] + 0.002

In [78]:
gu.head()

,gu,X,Y,Y1,Y2,Y3
0,종로구,126.977321,37.594917,37.594917,37.595917,37.596917
1,중구,126.995968,37.560144,37.560144,37.561144,37.562144
2,용산구,126.979907,37.531385,37.531385,37.532385,37.533385
3,성동구,127.041059,37.551030,37.551030,37.552030,37.553030
4,광진구,127.085744,37.546706,37.546706,37.547706,37.548706


In [79]:
out_place_num = pd.merge(out_num, gu, how='inner', on="gu")
in_place_num = pd.merge(in_num, gu, how='inner', on="gu")
tour_place_num = pd.merge(tour_num, gu, how='inner', on="gu")

In [80]:
import requests
import json


# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [124]:
import folium

m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles = 'cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구',
    style_function = lambda x: {'fillColor':'#2FA599' ,
    							'color':'#2FA599',
                                'weight' :'1',
                                }
).add_to(m)


for lat, long, num in zip(out_place_num['Y'], out_place_num['X'], out_place_num["num"]):
    folium.CircleMarker([lat, long], color="coral",fill ="coral", radius = int(num)/2, popup = num, fill_opacity=0.3, weight=1).add_to(m)

for lat, long, num in zip(in_place_num['Y'], in_place_num['X'], in_place_num["num"]):
    folium.CircleMarker([lat, long], color="yellow",fill ="yellow", radius = int(num)/2, popup = num, fill_opacity=0.3, weight=1).add_to(m)

for lat, long, num in zip(tour_place_num['Y'], tour_place_num['X'], tour_place_num["num"]):
    folium.CircleMarker([lat, long], color="#8F92D5",fill ="#8F92D5", radius = int(num)/2, popup = num, fill_opacity=0.3, weight=1).add_to(m)

m

In [123]:
import folium

m = folium.Map(
    location=[37.559819, 126.963895],
    zoom_start=11, 
    tiles = 'cartodbpositron'
)

folium.GeoJson(
    seoul_geo,
    name='지역구',
    style_function = lambda x: {'fillColor':'blue' ,
    							'color':'blue',
                                'weight' :'1',
                                }
).add_to(m)


for lat, long, num in zip(out_place_num['Y'], out_place_num['X'], out_place_num["num"]):
    folium.CircleMarker([lat, long], color="red",fill ="red", radius = int(num)/2, popup = num, fill_opacity=0.3, weight=1).add_to(m)

for lat, long, num in zip(in_place_num['Y'], in_place_num['X'], in_place_num["num"]):
    folium.CircleMarker([lat, long], color="yellow",fill ="yellow", radius = int(num)/2, popup = num, fill_opacity=0.3, weight=1).add_to(m)

for lat, long, num in zip(tour_place_num['Y'], tour_place_num['X'], tour_place_num["num"]):
    folium.CircleMarker([lat, long], color="white",fill ="white", radius = int(num)/2, popup = num, fill_opacity=0.3, weight=1).add_to(m)

m